

# 第4课：值迭代与策略迭代（Value Iteration & Policy Iteration）

---

## Part 1：值迭代算法（Value Iteration）

值迭代是基于\*\*贝尔曼最优方程（Bellman Optimality Equation）\*\*的迭代方法，其核心思想是：

> 利用 Bellman 最优方程反复更新状态值函数 $V(s)$，直到收敛为 $V^*(s)$，然后从中提取最优策略。

---

### 🔸 算法形式（同步更新）：

初始化 $V_0(s)$ 为任意值（例如 0），然后迭代如下更新：

$$
V_{k+1}(s) = \max_{a} \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V_k(s') \right]
$$

直到 $|V_{k+1}(s) - V_k(s)| < \epsilon$，即值函数收敛。

---

### 🔸 最终策略提取：

值函数收敛后，通过贪婪原则获得策略：

$$
\pi^*(s) = \arg\max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V^*(s') \right]
$$

---

### 🔸 伪代码：

```python
# Value Iteration
Initialize V(s) arbitrarily
Repeat:
    Δ ← 0
    For each state s:
        v ← V(s)
        V(s) ← max_a [ R(s,a) + γ Σ_s' P(s'|s,a) V(s') ]
        Δ ← max(Δ, |v - V(s)|)
Until Δ < θ (convergence threshold)
Return optimal policy π(s) = argmax_a [ R(s,a) + γ Σ_s' P(s'|s,a) V(s') ]
```

---

### 🔸 特点：

| 优点             | 缺点                        |
| -------------- | ------------------------- |
| 易于实现，快速逼近最优值函数 | 每次都需要执行完整的“最大化”操作（策略隐式更新） |
| 不需要明确策略更新步骤    | 对大规模状态空间，计算量大             |




## Part 2：策略迭代算法（Policy Iteration）

策略迭代是一种**显式维护和更新策略**的方法，交替进行两个阶段：

1. **策略评估（Policy Evaluation）**：在固定策略 $\pi$ 下求解 $V^\pi(s)$
2. **策略改进（Policy Improvement）**：对所有状态执行贪婪更新，得到新策略 $\pi'(s)$

重复这两个步骤直到策略收敛。

---

### 步骤详解：

#### ① 策略评估：

给定策略 $\pi$，求解其值函数：

$$
V^\pi(s) = \sum_{a} \pi(a|s) \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V^\pi(s') \right]
$$

可以：

* 精确求解线性方程组
* 或使用迭代逼近（如 TD(0) 或近似方法）

#### ② 策略改进：

$$
\pi_{\text{new}}(s) = \arg\max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V^\pi(s') \right]
$$

---

### 伪代码：

```python
# Policy Iteration
Initialize policy π(s) arbitrarily
Repeat:
    Evaluate V^π by solving Bellman equation for π
    For each state s:
        π_new(s) ← argmax_a [ R(s,a) + γ Σ_s' P(s'|s,a) V^π(s') ]
    If π_new == π:
        break
    Else:
        π ← π_new
Return π
```

---

### 特点：

| 优点         | 缺点                 |
| ---------- | ------------------ |
| 通常收敛快，策略稳定 | 每轮评估开销大（需要精确或近似求解） |
| 每轮策略明确改进   | 策略评估若不收敛则可能不稳定     |



## Part 3：截断策略迭代算法（Truncated Policy Iteration）

为了兼顾值迭代的**快速更新**和策略迭代的**策略改进效率**，提出了**截断策略迭代**（也叫 modified policy iteration）。

---

### 基本思想：

> 在策略迭代中，策略评估不需要完全收敛，只执行 **有限轮迭代更新**（如 n 步）后就进行策略改进。

这样就能：

* 节省每轮的计算时间
* 更快进入策略更新阶段

---

### 算法结构：

```python
# Truncated Policy Iteration
Initialize π(s), V(s)
Repeat:
    # Policy Evaluation with n iterations
    For i in range(n):
        For each state s:
            V(s) ← Σ_a π(a|s) [ R(s,a) + γ Σ_s' P(s'|s,a) V(s') ]

    # Policy Improvement
    For each state s:
        π(s) ← argmax_a [ R(s,a) + γ Σ_s' P(s'|s,a) V(s') ]
Until π converges
Return π
```

n 通常为 1\~5，根据任务大小调节。

---

### 特点：

| 优点               | 缺点             |
| ---------------- | -------------- |
| 较值迭代更稳定，较策略迭代更高效 | 参数 $n$ 需要调节    |
| 实际中常用于平衡开销和收敛速度  | 仍需存储完整的策略和价值函数 |



## 小结

| 算法     | 更新目标    | 每步计算         | 收敛速度   | 是否显式维护策略 |
| ------ | ------- | ------------ | ------ | -------- |
| 值迭代    | $V^*$   | 对所有状态执行最大化   | 快（通常）  | 否（策略后提取） |
| 策略迭代   | $\pi^*$ | 策略评估 + 贪婪更新  | 稳定但可能慢 | 是        |
| 截断策略迭代 | $\pi^*$ | 策略评估不完全 + 更新 | 平衡     | 是        |